In [ ]:
# run this cell first
reload_lamb()
from lamb.meta import Partial
lang.set_system(lang.topdown_presup)
lang.get_system()

# Presuppositions via partiality

The Heim & Kratzer approach to presuppositions works by allowing denotations and functions to be partially defined: the definedness conditions characterize the conditions under which the presupposition is satisfied.  Strictly speaking, it is *only* denotations and functions that involve partiality in this system, that is, not ordinary formulas (to the extent they exist in H&K).  However, it is impossible to implement this without having some representation for partiality in the metalanguage.  This is accomplished with the class `meta.Partial`.  

Unlike most subclasses of `TypedExpr`, instances of `Partial` do not have their own unique type -- their type is determined from their content.  Orthogonally to the type system, they layer a typed expression of type `t` on top of that content object, corresponding to the partiality.  They are drawn with two rows between bars.  The following example is the expression of type `e` that refers to `John`, but has a definedness condition that that individual be salient.  In the strict Heim and Kratzer sense, such formulas don't really appear on their own, since definedness conditions like this need to be associated with some denotation, as in the second cell below.

In [ ]:
Partial(te("John_e"), te("Salient_<e,t>(John_e)"))

In [ ]:
%%lamb
||john|| = Partial(John_e, Salient(John_e))

The case where you get partiality in a formula is for partially defined functions.  The notation is basically the same: use `Partial` in the body of a function.  For example, if you take the noun `dog` to presuppose animacy of its argument, you would write that as follows:

In [ ]:
%%lamb
||dog|| = L x_e : Partial(Dog(x), Animate(x))

More examples (mixed with things I'll use later):

In [ ]:
%%lamb
||cat|| = L x_e : Cat(x)
||the|| = L f_<e,t> : Partial((Iota x_e : f(x)), (ExistsExact x_e : f(x)))
||everyone|| = L f_<e,t> : Partial((Forall x_e : Sentient(x) ==> f(x)), (Exists x_e : Sentient(x)))
||a|| = L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)
||alfonso|| = Alfonso_e
||john|| = Partial(John_e, Salient(John_e))
testf = L f_<e,t> : L x_e : Partial(f(x), f(x))
||saw|| = L y_e : L x_e : Saw(x,y)
||POSS|| = L f_<e,<e,t>> : L x_e : Partial((Iota y_e : f(x)(y)), (ExistsExact y : f(x)(y)))
||brother|| = L x_e : L y_e : BrotherOf(y,x)

Reduction of a partially defined function is reasonably straightforward: the variable is replaced as expected in both the ordinary body of the function and in the partiality condition.

In [ ]:
dog.content(alfonso.content).reduce()

Here's another simple case, where there's really only one presuppositional item in a derivation.  (Here I'm showing full composition directly, which I'll return to.)

In [ ]:
alfonso * (saw * (alfonso * (POSS * brother)))

The above case is easy, but reduction with partiality gets complicated very quickly.  For example, what happens when the argument to a function is partial?  These are cases that as humans working with Heim & Kratzer pseudo-formulas in our head we can usually just get automatically, but they are not at all straightforward to think through mechanically.  Here is an example using the presupposition entry for `dog` above.  The partiality condition resulting from application gets lodged in a subformula under the existential quantifier, which actually binds into the partiality condition.  While this isn't necessarily the true analysis of `dog`, this situation might easily arise e.g. with relative clauses containing presupposition triggers.

In [ ]:
a.content(dog.content).reduce_all()

Simplifying this requires some semantics for the metalanguage (i.e. a formal understanding of what partiality should amount to in expressions like this).  The projection problem to some degree rests on this as well.  What the lambda notebook implements for this composition system is a fairly strict notion of partiality (that sometimes does odd things).  It is important to be clear that this is **not** a three-valued logic, and will behave quite differently from the usual three-valued logics used for projection in a range of cases.

The result of projecting the partiality out of the subformula above can be seen by composing the two NL items together.  As a rule, this sort of inference is not automatically triggered in the metalanguage ever, but is always triggered by composition (following the Heim and Kratzer treatment of partiality, I would argue).  This could be forced in the metalanguage by calling `calculate_partiality()` on a `TypedExpr`.

In [ ]:
a * dog

Things are even less straightforward when multiple presuppositions interact, and you end up generating things that I think humans will often miss in their calculations.  For example, the above entries for `dog` and for `the` collide in the following rather odd (but correct, for this partial semantics) result:

In [ ]:
the * dog

Part of what's going on here is the flatting of partiality conditions within partiality conditions into a rather strange result; the first conjunct is just the simple definite's presupposition, the second (universal) results from the partiality condition on the partiality condition, and the third results from the partiality condition on Iota.

In [ ]:
the.content(dog.content).reduce_all() #.condition.calculate_partiality()

This can sometimes be smoothed out be duplicating a partiality condition as an entailment, but the result is often still weird.

In [ ]:
%%lamb
||bachelor|| = L x_e : Partial(Unmarried(x) & Male(x) & Adult(x), Adult(x))

In [ ]:
the * bachelor

In [ ]:
everyone * bachelor # "everyone is a bachelor"

In [ ]:
(everyone * (saw * john))

In [ ]:
(everyone * (lang.Binder(1) * (everyone * (lang.Binder(2) * (lang.Trace(1) * (saw * lang.Trace(2)))))))

Next I give some more complicated examples involving binding of possessive pronouns:

    (1) John likes his_5 brother.
    (2) Every_6 student who_5 likes his_5 brother likes his_6 sister.
  
The resulting presuppositions are reasonable, but quite complicated and not obviously perfect.  I suggest you consider partiality a placeholder solution to the projeciton problem.

In [ ]:
male_prop = %te L x_e : Male_<e,t>(x)
himi = lang.PresupPronoun.index_factory("him", male_prop)
himi(5)

Side-note: notice that binding a pronoun of this sort always generates a partial function.

In [ ]:
%%lamb
||john|| = Partial(John_e, Salient(John_e))
||student|| = L x_e : Student(x)
||likes|| = L x_e : L y_e : Likes(y,x)
||brother|| = L x_e : L y_e : BrotherOf(y,x)
||sister|| = L x_e : L y_e : SisterOf(y,x)
||every|| = L f_<e,t> : L g_<e,t> : Partial((Forall x_e : f(x) ==> g(x)), (Exists x_e : f(x)))

||POSS|| = L f_<e,<e,t>> : L x_e : Partial((Iota y_e : f(x)(y)), (ExistsExact y : f(x)(y)))

In [ ]:
(john * (likes * (himi(5) * (POSS * brother))))

In [ ]:
# compute the subject: "[every student [5 [that t_5 likes his_5 brother]]]"
subject = (every * (student * (lang.Binder(5) * ((lang.Trace(5) * (likes * (himi(5) * (POSS * brother))))))))
subject

In [ ]:
# assume that the subject has QR'd in order to bind the possessive pronoun
subject * (lang.Binder(6) * (lang.Trace(6) * (likes * (himi(6) * (POSS * sister)))))